## Analysis

In [1]:
import A3_analysis
import A2_other
import A1_provider_prep

import pandas as pd
import geopandas as gpd
from calitp.sql import to_snakecase
from shared_utils import geography_utils, utils

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
# Read in no coverage maps
verizon = A1_provider_prep.load_verizon()

In [4]:
att = A1_provider_prep.load_att()

In [5]:
tmobile = A1_provider_prep.load_tmobile()

In [6]:
# Original provider map from FCC
att_og = gpd.read_parquet("./ATT.parquet")

In [7]:
# att_og.plot()

In [8]:
# att.plot()

### Which routes touch areas without data coverage among all 3 providers?

In [9]:
# Find routes that run in areas without coverage 
# Across all 3 providers
routes1 = A3_analysis.merge_all_providers()

In [10]:
# Load in original geometries of the routes.
one_dist_routes, multi_dist_routes, all_routes = A2_other.find_multi_district_routes()

In [11]:
# Create bins to see the median percentage of the route WITH coverage.
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
routes1["binned"] = pd.cut(routes1["median_percent_with_coverage"], bins)

In [12]:
f"There are {len(routes1)} routes that cross a zone w/o data coverage among ATT, T-Mobile, and Verizon out of {len(all_routes)} routes."

'There are 226 routes that cross a zone w/o data coverage among ATT, T-Mobile, and Verizon out of 2914 routes.'

### Filter out for low coverage routes
* Most routes (776 out of 865) have data coverage for 90-100% of the routes.
* Cut off based on bins.

In [13]:
routes1.binned.value_counts()

(90, 100]    140
(80, 90]      38
(70, 80]      29
(60, 70]       9
(50, 60]       6
(30, 40]       2
(40, 50]       2
(0, 10]        0
(10, 20]       0
(20, 30]       0
Name: binned, dtype: int64

In [14]:
routes1.median_percent_with_coverage.describe()

count   226.00
mean     89.26
std      12.88
min      36.00
25%      83.00
50%      95.00
75%      99.00
max     100.00
Name: median_percent_with_coverage, dtype: float64

In [15]:
# Filter out for routes where 75% or less of its length runs in a no data-coverage zone.
threshold = 80

In [16]:
low_coverage = (routes1.loc[routes1.median_percent_with_coverage < threshold]).reset_index(drop = True)

In [17]:
f"{len(low_coverage)} routes are considered low-data coverage"

'45 routes are considered low-data coverage'

### How many buses run through a route that has "low data coverage?"
* Find number of trips ran per these "low coverage" routes
* Find number of buses each agency from this "low coverage" dataframe owns.

In [18]:
final = A3_analysis.final_merge(low_coverage)

### Conclusions
* Change D-1 to full district names.

In [19]:
final.drop(columns = ["Geometry", "Binned"]).sort_values('Median Percent No Coverage', ascending = False)

,Agency,Long Route Name,District,Median Percent With Coverage,Median Percent No Coverage,Percentage Of Route Wo Coverage Att,Percentage Of Route Wo Coverage Verizon,Percentage Of Route Wo Coverage Tmobile,Total Trips By Route,Total Trips By Agency,Percentage Of Trips W Low Cell Service,Total Buses,Estimate Of Buses In Low Cell Zones
0,Trinity Transit,Route travels through all down-river communities between Weaverville and Willow Creek. Key route to the coast; connects with RTS service to Arcata/Eureka. 181 Trinity Transit,"D-1,D-1,D-2",36.00,64.00,70,62,64,4,17,23.53,7.00,1.65
38,Trinity Transit,Route between Weaverville and Hayfork. Connects to Redding Line in Douglas City. 178 Trinity Transit,D-2,39.00,61.00,77,47,61,4,17,23.53,7.00,1.65
28,Mendocino Transit Authority,The Coaster 230 Mendocino Transit Authority,D-1,45.00,55.00,55,4,68,4,106,3.77,35.00,1.32
40,Trinity Transit,Redding Line 288 Trinity Transit,D-2,46.00,54.00,54,13,55,4,17,23.53,7.00,1.65
42,Tulare County Regional Transit Agency,Porterville to Springville Loop c70 Tulare County Regional Transit Agency,D-6,53.00,47.00,2,47,55,4,511,0.78,18.00,0.14
41,Tulare County Area Transit,Porterville to Springville Loop 12181 Tulare County Area Transit,D-6,53.00,47.00,2,47,55,4,137,2.92,24.00,0.70
43,Yosemite Area Regional Transportation System,Sonora HWY 120 1094 Yosemite Area Regional Transportation System,D-10,55.00,45.00,45,39,45,2,21,9.52,10.00,0.95
31,Redwood Coast Transit,Route 199 Hiouchi / Gasquet 257 Redwood Coast Transit,D-1,55.00,45.00,34,45,45,6,63,9.52,14.00,1.33
44,Yosemite Area Regional Transportation System,Merced HWY 140 580 Yosemite Area Regional Transportation System,D-10,59.00,41.00,46,32,41,17,21,80.95,10.00,8.10
13,Capitol Corridor,Shuttle to San Francisco Transbay Terminal sf Capitol Corridor,D-4,60.00,40.00,38,40,40,18,45,40.00,65.00,26.00


In [20]:
f"Around {final['Estimate Of Buses In Low Cell Zones'].sum()} buses run through low data coverage routes."

'Around 167.3269767923633 buses run through low data coverage routes.'

In [21]:
f"There {final['Long Route Name'].nunique()} low data coverage routes."

'There 45 low data coverage routes.'

In [22]:
final.District.value_counts()

D-4            18
D-1             7
D-2             5
D-3             3
D-6             3
D-9             2
D-5             2
D-10            2
D-1,D-1,D-2     1
D-7             1
D-11            1
Name: District, dtype: int64

In [23]:
final.Agency.value_counts()

AC Transit                                                9
Trinity Transit                                           4
Golden Gate Bridge Highway and Transportation District    4
Eastern Sierra Transit Authority                          2
Santa Cruz Metropolitan Transit District                  2
Redwood Coast Transit                                     2
MUNI                                                      2
Yosemite Area Regional Transportation System              2
Eureka Transit Service                                    1
Gold Country Stage                                        1
Tulare County Regional Transit Agency                     1
Tulare County Area Transit                                1
Tahoe Truckee Area Regional Transportation                1
Tahoe Transportation                                      1
Sonoma County Transit                                     1
Arcata and Mad River Transit System                       1
San Diego Metropolitan Transit System   

#### Map

In [24]:
# Grab the names of routes are "low coverage."
# low_coverage_routes = low_coverage.long_route_name.unique().tolist()

In [25]:
# Get original geometry
#low_coverage_og_geometry = (
#    all_routes[all_routes["long_route_name"].isin(low_coverage_routes)]
#).reset_index(drop=True)

In [26]:
#tmobile_map = tmobile.explore(
 #   tiles="CartoDB positron",
#    width=800,
#    height=500,
#)

In [27]:
#tmobile_map = final.explore("Long Route Name",
#    m=tmobile_map,legend = False,
#)

# tmobile_map